In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/"Colab Notebooks"/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d plameneduardo/sarscov2-ctscan-dataset

 94% 217M/230M [00:02<00:00, 68.8MB/s]
100% 230M/230M [00:02<00:00, 107MB/s] 


In [ ]:
!unzip /content/sarscov2-ctscan-dataset.zip

Archive:  /content/sarscov2-ctscan-dataset.zip
  inflating: COVID/Covid (1).png     
  inflating: COVID/Covid (10).png    
  inflating: COVID/Covid (100).png   
  inflating: COVID/Covid (1000).png  
  inflating: COVID/Covid (1001).png  
  inflating: COVID/Covid (1002).png  
  inflating: COVID/Covid (1003).png  
  inflating: COVID/Covid (1004).png  
  inflating: COVID/Covid (1005).png  
  inflating: COVID/Covid (1006).png  
  inflating: COVID/Covid (1007).png  
  inflating: COVID/Covid (1008).png  
  inflating: COVID/Covid (1009).png  
  inflating: COVID/Covid (101).png   
  inflating: COVID/Covid (1010).png  
  inflating: COVID/Covid (1011).png  
  inflating: COVID/Covid (1012).png  
  inflating: COVID/Covid (1013).png  
  inflating: COVID/Covid (1014).png  
  inflating: COVID/Covid (1015).png  
  inflating: COVID/Covid (1016).png  
  inflating: COVID/Covid (1017).png  
  inflating: COVID/Covid (1018).png  
  inflating: COVID/Covid (1019).png  
  inflating: COVID/Covid (102).png   
  i

In [ ]:
import os
import random
import shutil

# Define the directory path containing the Covid-19 and Non-Covid-19 folders
data_dir = "/content"

# Define the directory path where you want to save the split dataset
split_dir = "/content/dataset"

# Define the train, validation, and test split ratios
train_ratio = 0.6
val_ratio = 0.3
test_ratio = 0.1
random.seed(123)

# Create the train, validation, and test directories if they don't exist
for split_name in ["train", "val", "test"]:
    for class_name in ["COVID", "non-COVID"]:
        dir_path = os.path.join(split_dir, split_name, class_name)
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

# Iterate through the Covid-19 and Non-Covid-19 folders and move the images to the corresponding split directories
for class_name in ["COVID", "non-COVID"]:
    source_folder = os.path.join(data_dir, class_name)
    files = os.listdir(source_folder)
    random.shuffle(files)

    # Calculate the number of images in each split
    num_train = int(train_ratio * len(files))
    num_val = int(val_ratio * len(files))
    num_test = int(test_ratio * len(files))

    # Move the images to the train split directory
    for i in range(num_train):
        file = files[i]
        source_path = os.path.join(source_folder, file)
        dest_path = os.path.join(split_dir, "train", class_name, file)
        shutil.copyfile(source_path, dest_path)

    # Move the images to the validation split directory
    for i in range(num_train, num_train+num_val):
        file = files[i]
        source_path = os.path.join(source_folder, file)
        dest_path = os.path.join(split_dir, "val", class_name, file)
        shutil.copyfile(source_path, dest_path)

    # Move the images to the test split directory
    for i in range(num_train, len(files)):
        file = files[i]
        source_path = os.path.join(source_folder, file)
        dest_path = os.path.join(split_dir, "test", class_name, file)
        shutil.copyfile(source_path, dest_path)


In [ ]:
from os.path import isfile, join
def findCommonDeep(path1, path2):
    commonfiles = []
    for dirpath1, _, filenames1 in os.walk(path1):
        for file in filenames1:
            for dirpath2, _, filenames2 in os.walk(path2):
                if file in filenames2 and isfile(join(dirpath2, file)):
                    commonfiles.append(file)

    print(len(commonfiles))

path1 = r'/content/dataset/val/non-COVID'
path2 = r'/content/dataset/test/non-COVID'
findCommonDeep(path1, path2)

368


In [ ]:
train_dir = 'dataset/train'
train_datagen = ImageDataGenerator(rescale = 1.0/255)

train = train_datagen.flow_from_directory(
    train_dir,
    target_size = (256, 202),
    class_mode = 'binary',
    batch_size = 32,
    shuffle = True,
)

Found 1488 images belonging to 2 classes.


In [ ]:
valid_dir = 'dataset/val'
valid_datagen = ImageDataGenerator(rescale = 1.0/255)

validate = train_datagen.flow_from_directory(
    valid_dir,
    target_size = (256, 202),
    class_mode = 'binary',
    batch_size = 32,
    shuffle = True,
)


Found 743 images belonging to 2 classes.


In [ ]:
test_dir = 'dataset/test'
test_datagen = ImageDataGenerator(rescale = 1.0/255)

test = train_datagen.flow_from_directory(
    test_dir,
    target_size = (256, 202),
    class_mode = 'binary',
    batch_size = 32,
    shuffle = True,
)

Found 993 images belonging to 2 classes.


#VGG 19

In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

pretrained_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 202, 3), pooling='max',classes=2)
for layer in pretrained_model.layers:
    layer.trainable = False
vgg19_model = Sequential()
vgg19_model.add(pretrained_model)
vgg19_model.add(Flatten())
vgg19_model.add(Dense(512, activation='relu'))
vgg19_model.add(Dense(1, activation='sigmoid'))

80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
vgg19_model.compile(optimizer=Adam(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
checkpoint_filepath6 = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath6,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
vgg19_model.fit(train, validation_data=validate, epochs=50, callbacks=[model_checkpoint_callback])

Epoch 1/50
47/47 [==============================] - 31s 369ms/step - loss: 0.6470 - accuracy: 0.6794 - val_loss: 0.4920 - val_accuracy: 0.7685
Epoch 2/50
47/47 [==============================] - 14s 291ms/step - loss: 0.4377 - accuracy: 0.7917 - val_loss: 0.4244 - val_accuracy: 0.8102
Epoch 3/50
47/47 [==============================] - 13s 287ms/step - loss: 0.4078 - accuracy: 0.8105 - val_loss: 0.3723 - val_accuracy: 0.8371
Epoch 4/50
47/47 [==============================] - 13s 284ms/step - loss: 0.3593 - accuracy: 0.8394 - val_loss: 0.4005 - val_accuracy: 0.8116
Epoch 5/50
47/47 [==============================] - 14s 294ms/step - loss: 0.3452 - accuracy: 0.8421 - val_loss: 0.3365 - val_accuracy: 0.8587
Epoch 6/50
47/47 [==============================] - 14s 290ms/step - loss: 0.2952 - accuracy: 0.8716 - val_loss: 0.3614 - val_accuracy: 0.8277
Epoch 7/50
47/47 [==============================] - 13s 286ms/step - loss: 0.2731 - accuracy: 0.8837 - val_loss: 0.3724 - val_accuracy: 0.8237

In [ ]:
loss, accuracy = vgg19_model.evaluate(test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

32/32 [==============================] - 6s 183ms/step - loss: 0.2666 - accuracy: 0.9063
Test loss: 0.2666
Test accuracy: 0.9063


In [ ]:
vgg19_model.load_weights(checkpoint_filepath6)

In [ ]:
loss, accuracy = vgg19_model.evaluate(test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

32/32 [==============================] - 6s 190ms/step - loss: 0.2419 - accuracy: 0.9094
Test loss: 0.2419
Test accuracy: 0.9094


#VGG16


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

pretrained_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 202, 3), pooling='max', classes=2)
for layer in pretrained_model.layers:
    layer.trainable = False
vgg16_model = Sequential()
vgg16_model.add(pretrained_model)
vgg16_model.add(Flatten())
vgg16_model.add(Dense(512, activation='relu'))
vgg16_model.add(Dense(1, activation='sigmoid'))

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
vgg16_model.compile(optimizer=Adam(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
checkpoint_filepath5 = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath5,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
vgg16_model.fit(train, validation_data=validate, epochs=50, callbacks=[model_checkpoint_callback])

Epoch 1/50
47/47 [==============================] - 15s 285ms/step - loss: 0.5964 - accuracy: 0.6915 - val_loss: 0.4393 - val_accuracy: 0.7900
Epoch 2/50
47/47 [==============================] - 13s 276ms/step - loss: 0.3957 - accuracy: 0.8165 - val_loss: 0.3751 - val_accuracy: 0.8291
Epoch 3/50
47/47 [==============================] - 13s 283ms/step - loss: 0.3650 - accuracy: 0.8300 - val_loss: 0.3164 - val_accuracy: 0.8641
Epoch 4/50
47/47 [==============================] - 13s 286ms/step - loss: 0.2851 - accuracy: 0.8824 - val_loss: 0.2959 - val_accuracy: 0.8694
Epoch 5/50
47/47 [==============================] - 13s 285ms/step - loss: 0.2760 - accuracy: 0.8931 - val_loss: 0.2891 - val_accuracy: 0.8816
Epoch 6/50
47/47 [==============================] - 13s 284ms/step - loss: 0.2942 - accuracy: 0.8703 - val_loss: 0.2802 - val_accuracy: 0.8843
Epoch 7/50
47/47 [==============================] - 12s 265ms/step - loss: 0.2334 - accuracy: 0.9086 - val_loss: 0.3242 - val_accuracy: 0.8493

In [ ]:
loss, accuracy = vgg16_model.evaluate(test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

32/32 [==============================] - 5s 163ms/step - loss: 0.2777 - accuracy: 0.9043
Test loss: 0.2777
Test accuracy: 0.9043


In [ ]:
vgg16_model.load_weights(checkpoint_filepath5)

In [ ]:
loss, accuracy = vgg16_model.evaluate(test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

32/32 [==============================] - 6s 192ms/step - loss: 0.2375 - accuracy: 0.9194
Test loss: 0.2375
Test accuracy: 0.9194
